# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
# df = data[['CustomerID','ProductName']].groupby(['Quantity']).sum()
df = pd.DataFrame(data.groupby(['CustomerID', 'ProductName']).Quantity.sum())
df

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table= pd.pivot_table(data, values="Quantity", index=["ProductName"], columns=['CustomerID'], aggfunc='sum', fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
step4 = pd.DataFrame(distances[33
                 ].sort_values(ascending=False)[1:6])

step4

,33
CustomerID,
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
step5 = df.loc[[3317,1072,3535,1920,3909]]

step5

Quantity
CustomerID ProductName                                
1072       Appetizer - Mini Egg Roll, Shrimp         1
           Appetizer - Sausage Rolls                 1
           Beans - Kidney, Canned                    1
           Beef - Montreal Smoked Brisket            1
           Beef - Striploin Aa                       1
...                                                ...
3909       Wine - Charddonnay Errazuriz              2
           Wine - Fume Blanc Fetzer                  1
           Wine - Magnotta, Merlot Sr Vqa            1
           Wine - Toasted Head                       1
           Wine - Valpolicella Masi                  1

[262 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
step6 = pd.DataFrame(step5.groupby(['ProductName']).Quantity.sum().sort_values(ascending=False))
step6

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3
...,...
Otomegusa Dashi Konbu,1
"Oregano - Dry, Rubbed",1
Onions - Cippolini,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
step7 = table.merge(step5, how='left', on='ProductName')
step7 = step7[step7[33]==0].sort_values('ProductName').index[:5]
step7

Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
       'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
       'Apricots - Halves'],
      dtype='object', name='ProductName')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
step8 = {}
lst =list(distances.columns)


for clientID in lst:

    step4 = distances[clientID].sort_values(ascending=False)[1:6]
    step5 = df.loc[clientID]
    step6 = pd.DataFrame(step5.groupby(['ProductName']).Quantity.sum().sort_values(ascending=False))
    step7 = table.merge(step5, how='left', on='ProductName')
    step7= step7[step7[clientID]==0].sort_values('ProductName')
        
    step8[clientID] = step7.index[:5]
    


In [12]:
step8

{33: Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
        'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
        'Apricots - Halves'],
       dtype='object', name='ProductName'),
 200: Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
        'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
        'Apricots - Dried'],
       dtype='object', name='ProductName'),
 264: Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
        'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
        'Apricots - Dried'],
       dtype='object', name='ProductName'),
 356: Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
        'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
        'Apricots - Dried'],
       dtype='object', name='ProductName'),
 412: Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
        'Appetizer - Mushroom Tart', 'Appetizer -

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
step8 = pd.DataFrame(step8)
step8.T

,0,1,2,3,4
33,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves
200,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
264,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
356,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
412,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves
...,...,...,...,...,...
97928,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Dried,Apricots Fresh
98069,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Dried,Apricots Fresh
98159,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
98185,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Apricots - Dried,Apricots Fresh,Arizona - Green Tea


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [16]:
distances1 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), 
                         index=table.columns, columns=table.columns)
distances1

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.479764,0.484514,0.525278,0.496086,0.500064,0.507963,0.504739,0.499300,0.500268,0.492619,...,0.494577,0.504692,0.487604,0.510055,0.534474,1.000000,0.525583,0.508923,0.492050,0.482921
98069,0.493197,0.534491,0.511845,0.484207,0.481259,0.484219,0.496892,0.490306,0.495330,0.506038,...,0.489353,0.499742,0.493332,0.484621,0.518025,0.525583,1.000000,0.511707,0.501063,0.498897
98159,0.486457,0.500253,0.503500,0.502204,0.502418,0.499195,0.500240,0.502118,0.510333,0.502173,...,0.505074,0.522726,0.500553,0.489177,0.506341,0.508923,0.511707,1.000000,0.504367,0.495580


In [18]:
def productreco(data):    
    for clientID in lst:

        step4 = data[clientID].sort_values(ascending=False)[1:6]
        step5 = df.loc[clientID]
        step6 = pd.DataFrame(step5.groupby(['ProductName']).Quantity.sum().sort_values(ascending=False))
        step7 = table.merge(step5, how='left', on='ProductName')
        step7= step7[step7[clientID]==0].sort_values('ProductName')

        step8 = step8[clientID] = step7.index[:5]
        
        return pd.DataFrame(step8)
    
productreco(distances1)
    

TypeError: Index does not support mutable operations